#### Distortion
- Question: What is the role of semantic map?  
- What happens when we put random values into semantic map and re-run our inference, does IoU change?

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [36]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import config
from src.distort import distort_semantic_maps, load_semantic_map, predict_with_distortion

In [38]:
project_root = '/Users/akerke/Documents/stacked-unets'
labels_dir = os.path.join(project_root,'output', 'etci_labels')
target_level = 1
output_dir = os.path.join(project_root,'output')

In [24]:
original_semantic_maps = load_semantic_map(labels_dir, target_level)
distorted_semantic_maps = distort_semantic_maps(original_semantic_maps) 
distorted_semantic_maps = [torch.Tensor(map_) for map_ in distorted_semantic_maps]

In [39]:
loaded_dict = np.load(f'{output_dir}/test_df.npy', allow_pickle=True).item()
test_df = pd.DataFrame.from_dict(loaded_dict)
test_df = test_df.reset_index(drop=True)

In [46]:
import albumentations as A
from torch.utils.data import DataLoader
from src.dataset import ETCIDataset


test_dataset = ETCIDataset(dataframe=test_df, split="test", transform=None)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [47]:
final_predictions, df_test, mean_iou = predict_with_distortion(
    test_loader=test_loader,
    df_test=test_df,
    level=target_level,
    distorted_semantic_maps=distorted_semantic_maps
)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.